In [ ]:
pip install transformers datasets torch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 56.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 74.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extension

In [ ]:
!pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 31.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 93.1 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


## Importing libraries

In [ ]:
import torch
from transformers import MBartForConditionalGeneration, MBartTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd

## Loading train and val dataset

In [ ]:
train_df = pd.read_csv('train.csv')
eval_df = pd.read_csv('test.csv')

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Example to train with a smaller subset, just for debugging:
#train_dataset = train_dataset.select(range(100))
#eval_dataset = eval_dataset.select(range(20))

datasets = DatasetDict({'train': train_dataset, 'eval': eval_dataset})


In [ ]:
print("Training dataset size:", len(train_dataset))
print("Evaluation dataset size:", len(eval_dataset))


Training dataset size: 107259
Evaluation dataset size: 24423


## Loading the mBART model and tokenizer


In [ ]:
from transformers import MBart50Tokenizer, MBartForConditionalGeneration

model_name = 'facebook/mbart-large-50'
tokenizer = MBart50Tokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)


/home/jovyan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Tokenizing the data

In [ ]:
# def preprocess_function(examples):
#     inputs = [ex for ex in examples['data']]
#     targets = [ex for ex in examples['correction']]

#     # Tokenize inputs normally
#     model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

#     # Set the source and target languages explicitly for German
#     tokenizer.src_lang = "de_DE"  # Set source language to German
#     tokenizer.tgt_lang = "de_DE"  # Set target language to German

#     # Tokenize targets with the target language set
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

#     model_inputs['labels'] = labels['input_ids']
#     return model_inputs

# tokenized_datasets = datasets.map(preprocess_function, batched=True)
def preprocess_function(examples):
    # Tokenize the input text and correction text with appropriate padding and truncation
    model_inputs = tokenizer(examples['data'], max_length=128, truncation=True, padding="max_length")
    # Set the source and target languages explicitly for German
    tokenizer.src_lang = "de_DE"  # Set source language to German
    tokenizer.tgt_lang = "de_DE"  # Set target language to German
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['correction'], max_length=128, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs
tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=['data', 'correction','annotations'])



Map:   0%|          | 0/107259 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/24423 [00:00<?, ? examples/s]

## Finetuning mBART

In [ ]:

# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     learning_rate=5e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_total_limit=3,
#     predict_with_generate=True
# )
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# Initialize training arguments
# Updated training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',               # Directory where the models will be saved
    evaluation_strategy='epoch',          # Evaluation is performed at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=4,        # Reduced batch size to decrease memory usage
    per_device_eval_batch_size=4,         # Consistent batch size for evaluation
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,                   # Only save the top 3 models (optional)
    fp16=True,                            # Enable mixed precision training
    gradient_accumulation_steps=4,        # Accumulate gradients to simulate larger batch sizes
    load_best_model_at_end=True,          # Load the best model at the end of training
    metric_for_best_model='loss',         # Use loss to determine the best model
    greater_is_better=False               # Lower loss indicates a better model
)

In [ ]:
# from datasets import load_metric
# import numpy as np

# # Load metrics
# accuracy_metric = load_metric("accuracy")
# f1_metric = load_metric("f1")
# bleu_metric = load_metric("sacrebleu")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     # Check if predictions or labels are empty or not as expected
#     if predictions.shape[0] == 0 or labels.shape[0] == 0:
#         return {"bleu": 0, "accuracy": 0, "f1": 0}
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Format predictions and labels for metrics computation
#     formatted_predictions = [pred.strip() for pred in decoded_preds]
#     formatted_labels = [[label.strip()] for label in decoded_labels]

#     # If you have classification outputs, ensure to adjust decoding as necessary
#     # and possibly convert outputs to integer labels if needed for accuracy and F1

#     # Compute metrics
#     bleu_result = bleu_metric.compute(predictions=formatted_predictions, references=formatted_labels)
#     accuracy_result = accuracy_metric.compute(predictions=formatted_predictions, references=[label[0] for label in formatted_labels])
#     f1_result = f1_metric.compute(predictions=formatted_predictions, references=[label[0] for label in formatted_labels], average='macro')  # Change averaging method as needed

#     return {
#         "bleu": bleu_result["score"],
#         "accuracy": accuracy_result["accuracy"],
#         "f1": f1_result["f1"]
#     }


In [ ]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_pred):
    metric = load_metric("sacrebleu")  # Example metric, adjust based on your task
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 used in labels as padding token with tokenizer.pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Format predictions and labels for the metric computation
    formatted_predictions = [pred.strip() for pred in decoded_preds]
    formatted_labels = [[label.strip()] for label in decoded_labels]

    # Compute the metric and return result
    result = metric.compute(predictions=formatted_predictions, references=formatted_labels)
    return {"bleu": result["score"]}


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'],
#     eval_dataset=tokenized_datasets['eval'],
#     tokenizer=tokenizer
# )

from transformers import DataCollatorForSeq2Seq
#data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True, truncation=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# # Assuming your training arguments and model have been initialized properly
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'],
#     eval_dataset=tokenized_datasets['eval'],  # Make sure this is the correct key for your evaluation dataset
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics
# )

from transformers import Trainer, AdamW, get_scheduler
from torch.optim.lr_scheduler import LambdaLR

class CustomTrainer(Seq2SeqTrainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        # Custom optimizer
        self.optimizer = AdamW(self.model.parameters(), lr=5e-5, weight_decay=0.01)

        # Custom LR scheduler: here we use a linear schedule with warmup
        self.lr_scheduler = get_scheduler(
            "linear",
            optimizer=self.optimizer,
            num_warmup_steps=500,  # Number of steps for the warmup
            num_training_steps=num_training_steps
        )

# Now you can use CustomTrainer instead of Seq2SeqTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/jovyan/.local/lib/python3.8/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:

print("Internal steps calculated by Trainer's DataLoader:", len(trainer.get_train_dataloader()))
from torch.utils.data import DataLoader

# Initialize the DataLoader with the correct collate function
train_loader = DataLoader(tokenized_datasets['train'], batch_size=8, collate_fn=data_collator)
print("Calculated steps from DataLoader:", len(train_loader))


Internal steps calculated by Trainer's DataLoader: 13408
Calculated steps from DataLoader: 13408


In [ ]:
from transformers import logging
logging.set_verbosity_debug()  # or set_verbosity_info() for less verbose output


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}")


Training on device: cuda


In [ ]:
import torch
torch.cuda.empty_cache()  # Clear unused memory

# Environment variable to manage memory allocation
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

In [ ]:
trainer.train()


Currently training with a batch size of: 8
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 107,259
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 40,224
  Number of trainable parameters = 610,879,488


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/tmp-checkpoint-500
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Configuration saved in ./results/tmp-checkpoint-500/config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Configuration saved in ./results/tmp-checkpoint-500/generation_config.json
Model weights saved in ./results/tmp-checkpoint-500/model.safetensors
tokenizer config file saved in ./results/tmp-checkpoint-500/tokenizer_config.json
Special tokens file

OutOfMemoryError: CUDA out of memory. Tried to allocate 17.17 GiB. GPU 0 has a total capacty of 47.54 GiB of which 15.41 GiB is free. Process 2116040 has 32.12 GiB memory in use. Of the allocated memory 24.09 GiB is allocated by PyTorch, and 7.72 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Evaluation

In [ ]:
results = trainer.evaluate()
print(results)


## GEC Model

In [ ]:
def generate_corrected_sentences(input_texts):
    # Tokenize the input texts
    inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True)

    # Generate the corrected sequences
    output_sequences = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

    # Decode the sequences to get the corrected sentences
    corrected_texts = [tokenizer.decode(seq, skip_special_tokens=True) for seq in output_sequences]
    return corrected_texts

# Example usage
test_sentences = ["ich sind Jon", "Another badly sentence."]
corrected_sentences = generate_corrected_sentences(test_sentences)
print(corrected_sentences)
